# CPU Parallelism: SIMD and VLIW

You know that CPUs execute instructions. You may have heard that modern CPUs do billions of operations per second. But have you ever wondered: how do they actually achieve such throughput?

The answer is parallelism - doing multiple things at once. But there are different *kinds* of parallelism that work at different levels. This notebook builds up the intuition for two key forms: **SIMD** and **VLIW**.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch
import time

plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

torch.manual_seed(42)
np.random.seed(42)

## The Problem: Sequential Execution is Slow

Let's start with a concrete problem. Say we have 8 numbers and we want to double each one.

In [ ]:
# Our input data: 8 numbers
input_values = [10, 20, 30, 40, 50, 60, 70, 80]
print(f"Input: {input_values}")

In [ ]:
# The naive approach: process one at a time
output_values = []
for val in input_values:
    doubled = val * 2
    output_values.append(doubled)

print(f"Output: {output_values}")

This works, but let's think about what the CPU is actually doing. Each iteration requires:
1. Load the value from memory
2. Multiply by 2
3. Store the result

**What's a cycle?** A CPU operates on a clock - millions or billions of "ticks" per second. A **cycle** is one tick of this clock. Simple operations like addition or multiplication typically complete in 1 cycle. A 3 GHz CPU has 3 billion cycles per second.

If each operation takes 1 cycle, processing 8 values takes **24 cycles** (8 loads + 8 multiplies + 8 stores).

Let's simulate this to see the timeline:

In [ ]:
# Simulate sequential execution
def simulate_sequential(n_values):
    """Returns list of (cycle, operation, value_index)"""
    timeline = []
    cycle = 0
    for i in range(n_values):
        timeline.append((cycle, 'load', i))
        cycle += 1
        timeline.append((cycle, 'multiply', i))
        cycle += 1
        timeline.append((cycle, 'store', i))
        cycle += 1
    return timeline, cycle

timeline_seq, total_cycles_seq = simulate_sequential(8)
print(f"Sequential execution: {total_cycles_seq} cycles for 8 values")

In [ ]:
# Visualize the sequential timeline
def plot_timeline(timeline, title, max_cycles=None):
    fig, ax = plt.subplots(figsize=(14, 3))
    
    colors = {'load': '#3498db', 'multiply': '#e74c3c', 'store': '#2ecc71'}
    y_positions = {'load': 2, 'multiply': 1, 'store': 0}
    
    for cycle, op, idx in timeline:
        y = y_positions[op]
        rect = plt.Rectangle((cycle, y), 0.9, 0.8, 
                             facecolor=colors[op], edgecolor='black', linewidth=0.5)
        ax.add_patch(rect)
        ax.text(cycle + 0.45, y + 0.4, f'v{idx}', ha='center', va='center', fontsize=8)
    
    max_x = max_cycles if max_cycles else max(c for c, _, _ in timeline) + 2
    ax.set_xlim(-0.5, max_x)
    ax.set_ylim(-0.5, 3.5)
    ax.set_xlabel('Cycle')
    ax.set_yticks([0.4, 1.4, 2.4])
    ax.set_yticklabels(['Store', 'Multiply', 'Load'])
    ax.set_title(title)
    ax.grid(True, axis='x', alpha=0.3)
    
    # Legend
    patches = [mpatches.Patch(color=c, label=l) for l, c in colors.items()]
    ax.legend(handles=patches, loc='upper right')
    
    plt.tight_layout()
    return fig

plot_timeline(timeline_seq, f'Sequential Execution: {total_cycles_seq} cycles', max_cycles=26)
plt.show()

Look at all that wasted potential! At any given cycle, only ONE operation is happening. The multiply hardware sits idle while we load. The store hardware sits idle while we multiply.

**What if we could do all 8 multiplies at once?**

## SIMD: Single Instruction, Multiple Data

Here's a key insight: look at what we're doing to each value.

- value 0: multiply by 2
- value 1: multiply by 2
- value 2: multiply by 2
- ... and so on

We're doing **the same operation** on **different data**. This pattern is incredibly common:
- Adding two vectors element-wise
- Scaling all pixels in an image
- Computing activations in a neural network layer

What if we had hardware that could execute one instruction on multiple values simultaneously?

In [ ]:
# Conceptually: instead of 8 separate multiplies...
print("Sequential thinking:")
for i, val in enumerate(input_values):
    print(f"  Cycle {i}: multiply value[{i}] = {val} * 2 = {val * 2}")

In [ ]:
# ...we do all 8 at once!
print("SIMD thinking (all in ONE cycle):")
print(f"  multiply value[0..7] = {input_values}")
print(f"                      * [2, 2, 2, 2, 2, 2, 2, 2]")
print(f"                      = {[v*2 for v in input_values]}")

This is called **SIMD**: Single Instruction, Multiple Data.

- **Single Instruction**: One "multiply" command
- **Multiple Data**: Applied to 8 values simultaneously

The hardware has 8 ALUs (Arithmetic Logic Units) working in parallel:

In [ ]:
# Visualize SIMD hardware concept
fig, ax = plt.subplots(figsize=(12, 5))

# Draw 8 ALUs side by side
alu_width = 1.2
alu_height = 1.0
spacing = 0.3

for i in range(8):
    x = i * (alu_width + spacing)
    
    # Input arrow
    ax.annotate('', xy=(x + alu_width/2, 3.5), xytext=(x + alu_width/2, 4.2),
                arrowprops=dict(arrowstyle='->', color='#3498db', lw=2))
    ax.text(x + alu_width/2, 4.4, f'{input_values[i]}', ha='center', fontsize=10, color='#3498db')
    
    # ALU box
    rect = FancyBboxPatch((x, 2.5), alu_width, alu_height, 
                          boxstyle="round,pad=0.05", facecolor='#e74c3c', edgecolor='black')
    ax.add_patch(rect)
    ax.text(x + alu_width/2, 3.0, f'ALU {i}', ha='center', va='center', fontsize=9, color='white', fontweight='bold')
    
    # Operation label
    ax.text(x + alu_width/2, 2.1, 'x2', ha='center', fontsize=9)
    
    # Output arrow
    ax.annotate('', xy=(x + alu_width/2, 1.0), xytext=(x + alu_width/2, 1.8),
                arrowprops=dict(arrowstyle='->', color='#2ecc71', lw=2))
    ax.text(x + alu_width/2, 0.7, f'{input_values[i]*2}', ha='center', fontsize=10, color='#2ecc71')

# Title and labels
ax.text(5.5, 5.2, 'SIMD: 8 ALUs execute the SAME instruction on DIFFERENT data', 
        ha='center', fontsize=12, fontweight='bold')
ax.text(5.5, 4.8, '(All in ONE cycle!)', ha='center', fontsize=10, style='italic')

ax.set_xlim(-0.5, 12)
ax.set_ylim(0, 5.5)
ax.axis('off')
plt.tight_layout()
plt.show()

### VLEN: The Vector Length

How many values can we process at once? This is called the **vector length** or **VLEN**.

Different CPUs have different VLEN:
- Intel SSE: 4 floats (128 bits)
- Intel AVX: 8 floats (256 bits)
- Intel AVX-512: 16 floats (512 bits)
- Our challenge machine: **VLEN = 8**

In [ ]:
# | export
VLEN = 8  # Our challenge machine's vector length
print(f"VLEN = {VLEN}")
print(f"One vector instruction processes {VLEN} values at once")

### What About Non-Multiples of VLEN?

Real data rarely comes in perfect multiples of 8. What happens if we have 9 elements?

The answer: we need **tail handling**. The last partial batch (elements 8) still uses a vector instruction, but with a **mask** that only processes the valid elements:

- Elements 0-7: Full vector operation (all 8 lanes active)
- Element 8: Partial vector operation (only 1 lane active, 7 lanes masked off)

This is handled automatically by modern vector ISAs, but it's worth knowing: if your data size is just slightly over a VLEN multiple (e.g., 9 or 17 elements), you still pay for a full extra vector instruction.

### The SIMD Speedup

Let's see how SIMD changes our timeline. Instead of 8 separate multiply instructions, we have ONE vector multiply:

In [ ]:
# Simulate SIMD execution (still sequential load-compute-store, but vectorized)
def simulate_simd_simple(n_values, vlen=8):
    """SIMD version: process VLEN values per instruction"""
    timeline = []
    cycle = 0
    
    for chunk_start in range(0, n_values, vlen):
        chunk_end = min(chunk_start + vlen, n_values)
        # Vector load
        for i in range(chunk_start, chunk_end):
            timeline.append((cycle, 'load', i))
        cycle += 1
        # Vector multiply
        for i in range(chunk_start, chunk_end):
            timeline.append((cycle, 'multiply', i))
        cycle += 1
        # Vector store
        for i in range(chunk_start, chunk_end):
            timeline.append((cycle, 'store', i))
        cycle += 1
    
    return timeline, cycle

timeline_simd, total_cycles_simd = simulate_simd_simple(8)
print(f"SIMD execution: {total_cycles_simd} cycles for 8 values")
print(f"Speedup: {total_cycles_seq / total_cycles_simd:.1f}x")

In [ ]:
# Visualize SIMD timeline - all 8 values in each operation
def plot_simd_timeline(timeline, title, vlen=8):
    fig, ax = plt.subplots(figsize=(14, 3))
    
    colors = {'load': '#3498db', 'multiply': '#e74c3c', 'store': '#2ecc71'}
    y_positions = {'load': 2, 'multiply': 1, 'store': 0}
    
    # Group by cycle and operation
    from collections import defaultdict
    grouped = defaultdict(list)
    for cycle, op, idx in timeline:
        grouped[(cycle, op)].append(idx)
    
    for (cycle, op), indices in grouped.items():
        y = y_positions[op]
        width = 0.9
        rect = plt.Rectangle((cycle, y), width, 0.8, 
                             facecolor=colors[op], edgecolor='black', linewidth=1)
        ax.add_patch(rect)
        label = f'v0-v{len(indices)-1}' if len(indices) > 1 else f'v{indices[0]}'
        ax.text(cycle + width/2, y + 0.4, label, ha='center', va='center', fontsize=9)
    
    max_x = max(c for c, _, _ in timeline) + 2
    ax.set_xlim(-0.5, 26)
    ax.set_ylim(-0.5, 3.5)
    ax.set_xlabel('Cycle')
    ax.set_yticks([0.4, 1.4, 2.4])
    ax.set_yticklabels(['Store', 'Multiply', 'Load'])
    ax.set_title(title)
    ax.grid(True, axis='x', alpha=0.3)
    
    plt.tight_layout()
    return fig

plot_simd_timeline(timeline_simd, f'SIMD Execution: {total_cycles_simd} cycles (vs {total_cycles_seq} sequential)')
plt.show()

Whoa! We went from 24 cycles to 3 cycles - an **8x speedup**. This is exactly VLEN.

But wait... look at the timeline again. At cycle 0, only the load hardware is working. At cycle 1, only the multiply hardware is working. At cycle 2, only the store hardware is working.

**These are different pieces of hardware. Why can't they work simultaneously?**

### Real-World SIMD: What NumPy Does Under the Hood

When you write vectorized NumPy code, it's using SIMD instructions:

In [ ]:
# NumPy's vectorized operations use SIMD under the hood
arr = np.array([10, 20, 30, 40, 50, 60, 70, 80], dtype=np.int32)
result = arr * 2  # This compiles to SIMD instructions!
print(f"NumPy: {arr} * 2 = {result}")

In [ ]:
# Let's see the difference with a larger array
large_arr = np.random.randn(1_000_000).astype(np.float32)

# Python loop (no SIMD)
def python_loop(arr):
    result = []
    for x in arr:
        result.append(x * 2)
    return result

# Time both approaches
start = time.time()
_ = python_loop(large_arr.tolist())
python_time = time.time() - start

start = time.time()
_ = large_arr * 2
numpy_time = time.time() - start

print(f"Python loop: {python_time*1000:.1f} ms")
print(f"NumPy (SIMD): {numpy_time*1000:.2f} ms")
print(f"Speedup: {python_time/numpy_time:.0f}x")

The massive speedup comes from multiple factors working together:

1. **SIMD instructions**: NumPy operations compile to vector instructions that process 4, 8, or 16 values per CPU cycle
2. **Avoiding Python overhead**: No interpreter dispatch, type checking, or object boxing per element
3. **Memory locality**: Contiguous arrays enable efficient cache usage and prefetching
4. **Compiled C code**: NumPy's core loops are pre-compiled, not interpreted

SIMD is a significant contributor, but even without SIMD, avoiding Python's per-element overhead would give substantial speedups.

**This is why "vectorize your code" is such important advice.** When you write a Python loop over array elements, you're throwing away all of these optimizations.

### Practice: Vectorize This Loop

Convert the following loop to use NumPy's vectorized operations:

In [ ]:
# Given: compute (a + b) * c for each element
a = np.array([1, 2, 3, 4, 5, 6, 7, 8])
b = np.array([10, 20, 30, 40, 50, 60, 70, 80])
c = np.array([2, 2, 2, 2, 2, 2, 2, 2])

# The slow way:
result_loop = []
for i in range(len(a)):
    result_loop.append((a[i] + b[i]) * c[i])
result_loop = np.array(result_loop)
print(f"Loop result: {result_loop}")

In [ ]:
# Your vectorized solution:
test_result_vectorized = (a + b) * c  # Replace with your answer

# Verify
assert np.allclose(test_result_vectorized, result_loop), "Results don't match!"
print(f"Vectorized result: {test_result_vectorized}")
print("Correct!")

## Vector Registers and vload/vstore

To do SIMD operations, we need a place to hold multiple values at once. This is where **vector registers** come in.

### Scalar vs Vector Registers

- **Scalar register**: Holds ONE value (like a regular variable)
- **Vector register**: Holds VLEN values (like a small array)

In [ ]:
# Conceptual comparison
scalar_register = 42  # One value
print(f"Scalar register: {scalar_register}")

vector_register = [10, 20, 30, 40, 50, 60, 70, 80]  # VLEN=8 values
print(f"Vector register: {vector_register}")

In [ ]:
# Visualize the difference
fig, axes = plt.subplots(1, 2, figsize=(12, 3))

# Scalar register
ax = axes[0]
rect = plt.Rectangle((0.3, 0.3), 0.4, 0.4, facecolor='#3498db', edgecolor='black')
ax.add_patch(rect)
ax.text(0.5, 0.5, '42', ha='center', va='center', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_title('Scalar Register\n(holds 1 value)', fontsize=12)
ax.axis('off')

# Vector register
ax = axes[1]
for i in range(8):
    rect = plt.Rectangle((i * 0.12 + 0.02, 0.3), 0.11, 0.4, 
                         facecolor='#e74c3c', edgecolor='black')
    ax.add_patch(rect)
    ax.text(i * 0.12 + 0.075, 0.5, f'{(i+1)*10}', ha='center', va='center', fontsize=10)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_title(f'Vector Register\n(holds {VLEN} values)', fontsize=12)
ax.axis('off')

plt.tight_layout()
plt.show()

### vload and vstore: Moving Data In and Out

To work with vector registers, we need special instructions:

- **vload**: Load VLEN contiguous values from memory into a vector register
- **vstore**: Store VLEN values from a vector register to contiguous memory locations

The key word is **contiguous** - the values must be next to each other in memory.

In [ ]:
# Simulate memory and vload/vstore
memory = [0] * 20  # Memory with 20 slots
memory[4:12] = [10, 20, 30, 40, 50, 60, 70, 80]  # Our data at addresses 4-11

print("Memory layout:")
for i in range(0, 20, 4):
    chunk = memory[i:i+4]
    print(f"  addr {i:2d}-{i+3:2d}: {chunk}")

In [ ]:
# vload: Load 8 contiguous values starting at address 4
def vload(memory, start_addr, vlen=8):
    """Load VLEN contiguous values from memory"""
    return memory[start_addr : start_addr + vlen]

vector_reg = vload(memory, start_addr=4)
print(f"vload from addr 4: {vector_reg}")
print(f"  -> Loaded {len(vector_reg)} values in ONE instruction!")

In [ ]:
# vstore: Store 8 values to contiguous memory locations
def vstore(memory, start_addr, values, vlen=8):
    """Store VLEN values to contiguous memory"""
    for i in range(vlen):
        memory[start_addr + i] = values[i]

# Double our values
doubled = [v * 2 for v in vector_reg]
print(f"After doubling: {doubled}")

# Store to addresses 12-19
vstore(memory, start_addr=12, values=doubled)

print("\nMemory after vstore:")
for i in range(0, 20, 4):
    chunk = memory[i:i+4]
    print(f"  addr {i:2d}-{i+3:2d}: {chunk}")

### Why Contiguous Matters

Think of vload like reaching onto a bookshelf and grabbing 8 books at once. This only works if the books are next to each other!

If your data is scattered (non-contiguous), you can't use vload - you'd have to load each value individually, losing the SIMD benefit.

In [ ]:
# Visualize contiguous vs scattered
fig, axes = plt.subplots(1, 2, figsize=(14, 3))

# Contiguous - vload works!
ax = axes[0]
for i in range(16):
    color = '#e74c3c' if 4 <= i < 12 else '#ecf0f1'
    rect = plt.Rectangle((i * 0.06, 0.3), 0.055, 0.4, facecolor=color, edgecolor='black')
    ax.add_patch(rect)
    if 4 <= i < 12:
        ax.text(i * 0.06 + 0.0275, 0.5, f'{(i-3)*10}', ha='center', va='center', fontsize=8)
ax.annotate('', xy=(0.24, 0.25), xytext=(0.72, 0.25),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
ax.text(0.48, 0.1, 'ONE vload', ha='center', fontsize=10, color='green', fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_title('Contiguous data: vload grabs all 8 at once', fontsize=11)
ax.axis('off')

# Scattered - need individual loads
ax = axes[1]
scattered_indices = [0, 2, 5, 6, 9, 11, 13, 15]  # Non-contiguous!
for i in range(16):
    color = '#e74c3c' if i in scattered_indices else '#ecf0f1'
    rect = plt.Rectangle((i * 0.06, 0.3), 0.055, 0.4, facecolor=color, edgecolor='black')
    ax.add_patch(rect)
for j, i in enumerate(scattered_indices[:4]):
    ax.annotate('', xy=(i * 0.06 + 0.0275, 0.25), xytext=(i * 0.06 + 0.0275, 0.1),
                arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
ax.text(0.48, 0.02, '8 separate loads needed!', ha='center', fontsize=10, color='red', fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_title('Scattered data: cannot use vload', fontsize=11)
ax.axis('off')

plt.tight_layout()
plt.show()

This is why **data layout matters so much for performance**. If you're designing data structures for high-performance code, you want related values stored contiguously so SIMD can work efficiently.

## VLIW: Very Long Instruction Word

We've seen SIMD: doing the same operation on multiple data. But there's another form of parallelism hiding in plain sight.

Look back at our SIMD timeline. Even with SIMD, we're executing operations in sequence:
1. Cycle 0: vload (8 values)
2. Cycle 1: multiply (8 values)
3. Cycle 2: vstore (8 values)

But wait - **loads, arithmetic, and stores use different hardware circuits!**

In [ ]:
# A CPU has different "engines" - separate pieces of hardware
cpu_engines = {
    'ALU': 'Does arithmetic: add, multiply, shift, etc.',
    'Load Unit': 'Fetches data from memory into registers',
    'Store Unit': 'Writes data from registers to memory',
    'Flow Control': 'Handles jumps, branches, conditionals'
}

print("CPU has separate hardware engines:")
for engine, desc in cpu_engines.items():
    print(f"  {engine}: {desc}")

Here's the key insight: **while the ALU is computing, the Load Unit could be fetching the NEXT batch of data!**

This is called **VLIW: Very Long Instruction Word**. Instead of one operation per cycle, we pack multiple operations (using different hardware) into a single "instruction bundle."

In [ ]:
# Traditional: one operation per instruction
traditional_program = [
    "load v0 from addr 0",
    "multiply v0 by 2", 
    "store v0 to addr 8",
]
print("Traditional (3 cycles):")
for i, instr in enumerate(traditional_program):
    print(f"  Cycle {i}: {instr}")

### The Dependency Constraint

Here's the catch: not all operations can run in parallel. Consider:

```
load v0 from memory    # Step 1: Load data into v0
multiply v0 by 2       # Step 2: Use v0 (depends on step 1!)
store v0 to memory     # Step 3: Store v0 (depends on step 2!)
```

You can't multiply v0 before you've loaded it. You can't store the result before you've computed it. These are **data dependencies** - each step needs the result of the previous step.

So what CAN run in parallel? Operations on DIFFERENT data! If we're processing multiple batches, we can:
- Load batch N+1 (into v1) while we compute batch N (in v0)
- The load and compute use different registers, so no conflict!

In [ ]:
# VLIW: bundle multiple operations that use different hardware
vliw_instruction = {
    'load': 'load v1 from addr 8',      # Load Unit
    'alu': 'multiply v0 by 2',           # ALU
    'store': 'store v_prev to addr 0',   # Store Unit
}
print("VLIW instruction bundle (ALL in ONE cycle):")
for engine, op in vliw_instruction.items():
    print(f"  {engine:6s}: {op}")

In [ ]:
# Visualize VLIW - multiple engines working in parallel
fig, ax = plt.subplots(figsize=(12, 5))

engines = ['Load Unit', 'ALU', 'Store Unit']
colors = {'Load Unit': '#3498db', 'ALU': '#e74c3c', 'Store Unit': '#2ecc71'}

# Draw the engines
for i, engine in enumerate(engines):
    y = i * 1.5
    rect = FancyBboxPatch((0.5, y), 2, 1, boxstyle="round,pad=0.05",
                          facecolor=colors[engine], edgecolor='black')
    ax.add_patch(rect)
    ax.text(1.5, y + 0.5, engine, ha='center', va='center', 
            fontsize=12, fontweight='bold', color='white')

# Draw the instruction bundle
ax.annotate('', xy=(3, 2), xytext=(4.5, 3.5),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
ax.annotate('', xy=(3, 2), xytext=(4.5, 2),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
ax.annotate('', xy=(3, 2), xytext=(4.5, 0.5),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5))

# Instruction text
ax.text(5.5, 3.5, 'vload v1, addr', fontsize=10, va='center')
ax.text(5.5, 2, 'vmul v0, v0, 2', fontsize=10, va='center')
ax.text(5.5, 0.5, 'vstore addr, v_prev', fontsize=10, va='center')

# Bundle box
rect = plt.Rectangle((4.3, 0), 4, 4), 
ax.add_patch(plt.Rectangle((4.3, 0), 4, 4, fill=False, edgecolor='black', 
                           linestyle='--', linewidth=2))
ax.text(6.3, 4.3, 'ONE Instruction Bundle', ha='center', fontsize=11, fontweight='bold')
ax.text(6.3, -0.4, '(All execute in ONE cycle)', ha='center', fontsize=10, style='italic')

ax.set_xlim(0, 9)
ax.set_ylim(-1, 5)
ax.set_title('VLIW: Different engines work in parallel', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

### The Pipeline Pattern

With VLIW, we can create a **pipeline** where different stages of processing happen simultaneously on different data:

| Cycle | Load Unit | ALU | Store Unit |
|-------|-----------|-----|------------|
| 0 | Load batch 0 | (idle) | (idle) |
| 1 | Load batch 1 | Compute batch 0 | (idle) |
| 2 | Load batch 2 | Compute batch 1 | Store batch 0 |
| 3 | Load batch 3 | Compute batch 2 | Store batch 1 |
| ... | ... | ... | ... |

After the pipeline fills up, we're processing ONE BATCH PER CYCLE!

In [ ]:
# Simulate pipelined VLIW execution
def simulate_vliw_pipeline(n_batches):
    """VLIW with overlapping load/compute/store"""
    schedule = []  # (cycle, engine, batch)
    
    for batch in range(n_batches):
        # Each batch goes through 3 stages, but stages overlap!
        schedule.append((batch, 'load', batch))      # Load starts at cycle=batch
        schedule.append((batch + 1, 'compute', batch))  # Compute at cycle=batch+1
        schedule.append((batch + 2, 'store', batch))    # Store at cycle=batch+2
    
    total_cycles = n_batches + 2  # Pipeline fill + drain
    return schedule, total_cycles

schedule, total_cycles = simulate_vliw_pipeline(4)
print(f"Processing 4 batches with pipelined VLIW: {total_cycles} cycles")
print(f"\nWithout pipelining: 4 * 3 = 12 cycles")
print(f"Speedup from pipelining: {12/total_cycles:.1f}x")

In [ ]:
# Visualize the pipeline
def plot_pipeline(schedule, n_cycles, title):
    fig, ax = plt.subplots(figsize=(14, 4))
    
    colors = {'load': '#3498db', 'compute': '#e74c3c', 'store': '#2ecc71'}
    y_positions = {'load': 2, 'compute': 1, 'store': 0}
    
    for cycle, engine, batch in schedule:
        y = y_positions[engine]
        rect = plt.Rectangle((cycle, y), 0.9, 0.8, 
                             facecolor=colors[engine], edgecolor='black', linewidth=1, alpha=0.8)
        ax.add_patch(rect)
        ax.text(cycle + 0.45, y + 0.4, f'B{batch}', ha='center', va='center', 
                fontsize=10, fontweight='bold')
    
    ax.set_xlim(-0.5, n_cycles + 0.5)
    ax.set_ylim(-0.5, 3.5)
    ax.set_xlabel('Cycle', fontsize=11)
    ax.set_yticks([0.4, 1.4, 2.4])
    ax.set_yticklabels(['Store Unit', 'ALU', 'Load Unit'])
    ax.set_title(title, fontsize=12)
    ax.grid(True, axis='x', alpha=0.3)
    
    # Legend
    patches = [mpatches.Patch(color=c, label=l) for l, c in colors.items()]
    ax.legend(handles=patches, loc='upper right')
    
    plt.tight_layout()
    return fig

plot_pipeline(schedule, 7, 'VLIW Pipeline: All engines stay busy')
plt.show()

Look at cycles 2-4: **all three engines are working simultaneously!** This is the power of VLIW.

- Load Unit is fetching batch N+2
- ALU is computing batch N+1
- Store Unit is saving batch N

After the initial "ramp up," we achieve **one batch of work per cycle** instead of **three cycles per batch**.

## Engine Slots: How Many Ops Per Cycle?

So far we've talked about engines like they can only do one thing at a time. But real hardware often has **multiple slots** per engine - meaning you can do multiple operations of the same type per cycle.

Our challenge machine has these slot limits:

In [ ]:
# | export
SLOT_LIMITS = {
    "alu": 12,      # 12 scalar ALU operations per cycle
    "valu": 6,      # 6 vector ALU operations per cycle (each processes VLEN values!)
    "load": 2,      # 2 load operations per cycle
    "store": 2,     # 2 store operations per cycle
    "flow": 1,      # 1 flow control operation per cycle
}

print("Slot limits per cycle:")
for engine, slots in SLOT_LIMITS.items():
    print(f"  {engine:6s}: {slots:2d} slot(s)")

In [ ]:
# Visualize the slot capacity
fig, ax = plt.subplots(figsize=(14, 5))

engines = ['alu', 'valu', 'load', 'store', 'flow']
colors_eng = {'alu': '#e74c3c', 'valu': '#9b59b6', 'load': '#3498db', 
              'store': '#2ecc71', 'flow': '#f39c12'}
labels = {'alu': 'Scalar ALU', 'valu': 'Vector ALU', 'load': 'Load', 
          'store': 'Store', 'flow': 'Flow Control'}

y = 0
for engine in engines:
    slots = SLOT_LIMITS[engine]
    for s in range(slots):
        rect = plt.Rectangle((s * 0.8, y), 0.75, 0.8, 
                             facecolor=colors_eng[engine], edgecolor='black', alpha=0.7)
        ax.add_patch(rect)
    ax.text(-0.5, y + 0.4, f'{labels[engine]} ({slots})', ha='right', va='center', fontsize=10)
    y += 1.2

ax.set_xlim(-4, 10)
ax.set_ylim(-0.5, 6.5)
ax.set_xlabel('Slot', fontsize=11)
ax.set_title('Available slots per cycle\n(Each box = one operation)', fontsize=12)
ax.axis('off')
plt.tight_layout()
plt.show()

This creates an interesting optimization problem: **how do you pack operations to use all available slots?**

Look at the imbalance:
- We have 12 scalar ALU slots and 6 vector ALU slots (lots of compute!)
- But only 2 load and 2 store slots (limited memory bandwidth)

In [ ]:
# Example: packing operations into one cycle
example_instruction = {
    "valu": [
        ("*", "dest0", "src0", "src1"),  # Slot 0: multiply
        ("+", "dest1", "src2", "src3"),  # Slot 1: add
    ],
    "load": [
        ("vload", "reg0", "addr0"),      # Slot 0: vector load
        ("vload", "reg1", "addr1"),      # Slot 1: vector load
    ],
    "store": [
        ("vstore", "addr2", "reg2"),     # Slot 0: vector store
    ],
}

print("Example instruction bundle (ONE cycle):")
for engine, ops in example_instruction.items():
    print(f"\n  {engine}:")
    for i, op in enumerate(ops):
        print(f"    Slot {i}: {op}")

In [ ]:
# How efficiently are we using the slots?
def analyze_slot_usage(instruction):
    print("Slot usage analysis:")
    for engine in SLOT_LIMITS:
        if engine == 'debug':
            continue
        used = len(instruction.get(engine, []))
        available = SLOT_LIMITS[engine]
        utilization = used / available * 100
        bar = '#' * used + '.' * (available - used)
        print(f"  {engine:6s}: [{bar}] {used}/{available} ({utilization:.0f}%)")

analyze_slot_usage(example_instruction)

**The art of high-performance programming on VLIW machines is scheduling operations to maximize slot usage.** If you have lots of arithmetic to do but few loads, you'll be limited by how fast you can get data. If you have complex control flow, that single flow slot becomes the bottleneck.

## Combining SIMD and VLIW

Now we can put both pieces together:

- **SIMD** (horizontal parallelism): Do the same operation on VLEN values at once
- **VLIW** (vertical parallelism): Do multiple different operations in the same cycle

Combined, we can achieve massive throughput.

In [ ]:
# Let's calculate the theoretical throughput
vlen = 8
valu_slots = 6

values_per_valu_op = vlen  # Each vector op processes VLEN values
valu_ops_per_cycle = valu_slots
values_computed_per_cycle = values_per_valu_op * valu_ops_per_cycle

print(f"SIMD: Each vector operation processes {vlen} values")
print(f"VLIW: Can do {valu_slots} vector ALU operations per cycle")
print(f"Combined: {values_computed_per_cycle} values computed per cycle!")

In [ ]:
# Visualize what happens in ONE cycle with SIMD + VLIW
fig, ax = plt.subplots(figsize=(14, 8))

# Title
ax.text(7, 7.5, 'ONE CYCLE: SIMD + VLIW Combined', 
        ha='center', fontsize=14, fontweight='bold')

# Vector ALU slots (6 slots, each processing 8 values)
ax.text(0.5, 6.5, 'Vector ALU (6 slots):', fontsize=11, fontweight='bold')
for slot in range(6):
    y_base = 5.5 - slot * 0.9
    ax.text(0.5, y_base + 0.2, f'Slot {slot}:', fontsize=9)
    for v in range(8):
        rect = plt.Rectangle((2 + v * 0.6, y_base), 0.55, 0.5, 
                             facecolor='#e74c3c', edgecolor='black', alpha=0.7)
        ax.add_patch(rect)
        ax.text(2 + v * 0.6 + 0.275, y_base + 0.25, f'v{v}', 
                ha='center', va='center', fontsize=7)

# Load slots (2 slots)
ax.text(8, 6.5, 'Load (2 slots):', fontsize=11, fontweight='bold')
for slot in range(2):
    y_base = 5.5 - slot * 0.9
    for v in range(8):
        rect = plt.Rectangle((8 + v * 0.6, y_base), 0.55, 0.5, 
                             facecolor='#3498db', edgecolor='black', alpha=0.7)
        ax.add_patch(rect)

# Store slots (2 slots)
ax.text(8, 3.7, 'Store (2 slots):', fontsize=11, fontweight='bold')
for slot in range(2):
    y_base = 2.7 - slot * 0.9
    for v in range(8):
        rect = plt.Rectangle((8 + v * 0.6, y_base), 0.55, 0.5, 
                             facecolor='#2ecc71', edgecolor='black', alpha=0.7)
        ax.add_patch(rect)

# Summary
ax.text(7, 0.5, f'Total in ONE cycle: {6*8} computes + {2*8} loads + {2*8} stores = {6*8 + 2*8 + 2*8} value operations!',
        ha='center', fontsize=11, fontweight='bold', 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax.set_xlim(0, 14)
ax.set_ylim(0, 8)
ax.axis('off')
plt.tight_layout()
plt.show()

In a single cycle, our challenge machine can potentially:
- Compute 48 values (6 valu slots x 8 values each)
- Load 16 values (2 load slots x 8 values each)
- Store 16 values (2 store slots x 8 values each)

That's up to **80 value operations per cycle** when everything is perfectly scheduled!

### A Full Pipeline Example

Let's trace through a realistic example: processing 32 values (4 batches of 8).

In [ ]:
# Simulate a fully pipelined execution
def simulate_full_pipeline(n_values, vlen=8):
    n_batches = (n_values + vlen - 1) // vlen
    
    print(f"Processing {n_values} values in {n_batches} batches of {vlen}")
    print("\nPipeline execution:")
    print(f"{'Cycle':<8} {'Load':<15} {'Compute':<15} {'Store':<15}")
    print("-" * 53)
    
    total_cycles = n_batches + 2  # Fill + drain
    
    for cycle in range(total_cycles):
        load_batch = cycle if cycle < n_batches else '-'
        compute_batch = cycle - 1 if 0 <= cycle - 1 < n_batches else '-'
        store_batch = cycle - 2 if 0 <= cycle - 2 < n_batches else '-'
        
        load_str = f'Batch {load_batch}' if load_batch != '-' else '(idle)'
        compute_str = f'Batch {compute_batch}' if compute_batch != '-' else '(idle)'
        store_str = f'Batch {store_batch}' if store_batch != '-' else '(idle)'
        
        print(f"{cycle:<8} {load_str:<15} {compute_str:<15} {store_str:<15}")
    
    return total_cycles

cycles = simulate_full_pipeline(32)

In [ ]:
# Compare to non-pipelined execution
n_values = 32
n_batches = 4

sequential_cycles = n_batches * 3  # 3 stages per batch
pipelined_cycles = n_batches + 2    # Pipeline overhead

print(f"Sequential (no pipeline): {sequential_cycles} cycles")
print(f"Pipelined VLIW: {pipelined_cycles} cycles")
print(f"Speedup: {sequential_cycles / pipelined_cycles:.1f}x")
print(f"\nWith more batches, the speedup approaches 3x (the pipeline depth)")

## The Bottleneck Game

In theory, we have 48 compute values per cycle but only 16 load values per cycle. In practice, this means **memory bandwidth is often the bottleneck**, not compute.

Let's explore this with a concrete example.

In [ ]:
# Scenario: Element-wise multiply of two arrays, store result
# For each output element, we need:
# - 2 loads (one from each input array)
# - 1 multiply
# - 1 store

print("Operation: result[i] = a[i] * b[i]")
print("\nPer element:")
print("  Loads needed: 2 (a[i] and b[i])")
print("  Computes needed: 1 (multiply)")
print("  Stores needed: 1 (result[i])")

In [ ]:
# How many elements can we process per cycle?
load_slots = 2
store_slots = 2
valu_slots = 6
vlen = 8

# Memory perspective: 2 loads needed per element
# With 2 load slots, we can do 2 vloads = 16 values loaded
# But we need 2 loads per output element, so that's 8 output elements
loads_per_cycle = load_slots * vlen  # 16 values loaded
loads_per_output = 2  # Need 2 arrays
max_outputs_from_loads = loads_per_cycle // loads_per_output  # 8 outputs

# Compute perspective: 1 multiply per output element
# With 6 valu slots, we could do 6 vmuls = 48 values
computes_per_cycle = valu_slots * vlen  # 48 values
computes_per_output = 1
max_outputs_from_compute = computes_per_cycle // computes_per_output  # 48 outputs

# Store perspective: 1 store per output
stores_per_cycle = store_slots * vlen  # 16 values
stores_per_output = 1
max_outputs_from_stores = stores_per_cycle // stores_per_output  # 16 outputs

print("Throughput analysis for a[i] * b[i]:")
print(f"  From load capacity: {max_outputs_from_loads} outputs/cycle")
print(f"  From compute capacity: {max_outputs_from_compute} outputs/cycle")
print(f"  From store capacity: {max_outputs_from_stores} outputs/cycle")
print(f"\nBottleneck: {min(max_outputs_from_loads, max_outputs_from_compute, max_outputs_from_stores)} outputs/cycle")

In [ ]:
# Visualize the bottleneck
fig, ax = plt.subplots(figsize=(12, 5))

categories = ['Load\n(2 slots)', 'Compute\n(6 valu slots)', 'Store\n(2 slots)']
capacities = [max_outputs_from_loads, max_outputs_from_compute, max_outputs_from_stores]
colors_bar = ['#3498db', '#e74c3c', '#2ecc71']

bars = ax.bar(categories, capacities, color=colors_bar, edgecolor='black')

# Highlight the bottleneck
min_cap = min(capacities)
for bar, cap in zip(bars, capacities):
    if cap == min_cap:
        bar.set_edgecolor('red')
        bar.set_linewidth(3)

# Bottleneck line
ax.axhline(y=min_cap, color='red', linestyle='--', linewidth=2, label=f'Bottleneck: {min_cap} outputs/cycle')

ax.set_ylabel('Max outputs per cycle')
ax.set_title('For a[i] * b[i]: Load is the bottleneck!\n(We have compute capacity for 48 but can only load for 8)', fontsize=11)
ax.legend()

# Add value labels
for bar, cap in zip(bars, capacities):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
            str(cap), ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

**Key insight**: Even though we have 6 vector ALU slots (capable of 48 computations per cycle), we can only feed them with 2 load slots. For this workload, we're **memory-bound**, not compute-bound.

This is extremely common in practice. Modern CPUs have so much compute power that the challenge is often getting data to the compute units fast enough.

### Practice: Find the Bottleneck

In [ ]:
# Scenario: result[i] = (a[i] + b[i]) * (c[i] - d[i])
# Per output element:
# - Loads: 4 (a, b, c, d)
# - Computes: 3 (add, subtract, multiply)
# - Stores: 1

print("Operation: result[i] = (a[i] + b[i]) * (c[i] - d[i])")
print("\nYour task: Calculate the bottleneck")
print(f"  Load slots: {load_slots}, each loads {vlen} values")
print(f"  VALU slots: {valu_slots}, each computes {vlen} values")
print(f"  Store slots: {store_slots}, each stores {vlen} values")

In [ ]:
# Solution
loads_needed_per_output = 4
computes_needed_per_output = 3
stores_needed_per_output = 1

test_max_from_loads = (load_slots * vlen) // loads_needed_per_output
test_max_from_compute = (valu_slots * vlen) // computes_needed_per_output  
test_max_from_stores = (store_slots * vlen) // stores_needed_per_output

test_bottleneck = min(test_max_from_loads, test_max_from_compute, test_max_from_stores)

print(f"Max outputs from loads: {test_max_from_loads}")
print(f"Max outputs from compute: {test_max_from_compute}")
print(f"Max outputs from stores: {test_max_from_stores}")
print(f"\nBottleneck: {test_bottleneck} outputs/cycle")

# Verify
assert test_bottleneck == 4, f"Expected bottleneck of 4, got {test_bottleneck}"
print("Correct! Load is still the bottleneck (4 loads needed per output, only 2 load slots)")

### When Compute IS the Bottleneck

Sometimes compute can be the bottleneck - when you're doing lots of operations on data that's already loaded:

In [ ]:
# Scenario: Apply 10 sequential operations to data already in registers
# (e.g., a complex hash function with many steps)

# Data is loaded ONCE, then we do many operations
loads_per_output = 1
computes_per_output = 10  # Hash function has many steps
stores_per_output = 1

max_from_loads = (load_slots * vlen) // loads_per_output
max_from_compute = (valu_slots * vlen) // computes_per_output
max_from_stores = (store_slots * vlen) // stores_per_output

print("Scenario: Hash function with 10 sequential operations")
print(f"  Max from loads: {max_from_loads} outputs/cycle")
print(f"  Max from compute: {max_from_compute} outputs/cycle")
print(f"  Max from stores: {max_from_stores} outputs/cycle")
print(f"\nNow COMPUTE is the bottleneck! ({max_from_compute} < {max_from_loads})")

## Summary: Two Dimensions of Parallelism

We've explored two complementary forms of CPU parallelism:

### SIMD (Single Instruction, Multiple Data)
- **What**: Process VLEN values with one instruction
- **How**: Vector registers + vector operations (vload, vstore, valu)
- **Key requirement**: Data must be contiguous in memory
- **Speedup**: Up to VLEN (8x on our machine)

### VLIW (Very Long Instruction Word)
- **What**: Execute multiple operations using different hardware in one cycle
- **How**: Bundle operations for ALU, load, store, flow into one instruction
- **Key insight**: Different engines can work in parallel
- **Enables**: Pipelining (load batch N+1 while computing batch N)

### Combined Power
Our challenge machine can potentially do:
- 6 vector ALU ops x 8 values = 48 computes/cycle
- 2 vector loads x 8 values = 16 loads/cycle  
- 2 vector stores x 8 values = 16 stores/cycle

### The Bottleneck Reality
- Theoretical throughput is limited by the scarcest resource
- Often memory-bound (loads are precious!)
- Scheduling operations to maximize slot usage is the art of VLIW programming

In [ ]:
# | export
# Quick reference
print("Challenge Machine Quick Reference")
print("=" * 40)
print(f"VLEN = {VLEN} (values per vector operation)")
print("\nSlot limits per cycle:")
for engine, slots in SLOT_LIMITS.items():
    if engine != 'debug':
        print(f"  {engine:6s}: {slots:2d}")
print("\nMax throughput per cycle:")
print(f"  Compute: {SLOT_LIMITS['valu'] * VLEN} values (valu) + {SLOT_LIMITS['alu']} values (alu)")
print(f"  Load:    {SLOT_LIMITS['load'] * VLEN} values")
print(f"  Store:   {SLOT_LIMITS['store'] * VLEN} values")